In [3]:
import json
import pandas as pd
import re

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

#from transformers import AutoTokenizer #kills my kernal

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/meganmoore/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/meganmoore/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/meganmoore/nltk_data...


True

In [6]:
with open('../data/metadata_w_2020articles.json') as json_file:
    data = json.load(json_file)



In [7]:
data

{'bcbc6bb2-406e-11ee-a96e-33dec8f414a2': {'source': 'wp',
  'year': '2020',
  'article_text': 'President Trump on Monday said he was “proud” of Federal Reserve Chairman Jerome Powell’s handling of the economic turmoil brought on by the coronavirus pandemic, backing down from more than a year of criticizing his hand-picked Fed chief and threatening his job security.Trump praised Powell during a Monday evening briefing at the White House after the Fed that morning drastically expanded its efforts to protect the U.S. economy and financial markets amid the coronavirus pandemic.“I really think he’s caught',
  'title': '\n\t\tTrump shifts tone, says he’s ‘proud’ of Fed amid coronavirus turmoil\t'},
 'cfb4ce29-406e-11ee-a96e-33dec8f414a2': {'source': 'wp',
  'year': '2020',
  'article_text': 'U.S. intelligence reportedly indicates Iran’s government offered bounties to Taliban fighters to target U.S. and coalition forces in Afghanistan.A Pentagon briefing document said a foreign government pai